<a href="https://colab.research.google.com/github/mohdomer/CVIP-DataScience-Internship/blob/main/speech_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import glob
import soundfile
import os
import sys


In [2]:
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

In [3]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
from IPython.display import Audio

In [5]:
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [8]:
! wget https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio/download?datasetVersionNumber=1

--2023-11-16 18:10:33--  https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio/download?datasetVersionNumber=1
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fuwrfkaggler%2Fravdess-emotional-speech-audio%2Fversions%2F1%3Fresource%3Ddownload [following]
--2023-11-16 18:10:33--  https://www.kaggle.com/account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fuwrfkaggler%2Fravdess-emotional-speech-audio%2Fversions%2F1%3Fresource%3Ddownload
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘download?datasetVersionNumber=1’

download?datasetVer     [

In [9]:
Ravdessdata="audio_speech_actors_01-24"

In [11]:
Directory=os.listdir(Ravdessdata)
fileEmotion,filepath=[],[]
for dir in Directory:
    actor=os.listdir(Ravdessdata+dir)
    for file in actor:
        part=file.split('.')[0]
        part=part.split('-')
        fileEmotion.append(int(part[2]))
        filepath.append(Ravdessdata+dir+'/'+file)
emotion_df=pd.DataFrame(fileEmotion,columns=['Emotions'])
path_df=pd.DataFrame(filepath,columns=['Path'])
Ravdess_df=pd.concat([emotion_df,path_df],axis=1)

In [ ]:
Ravdess_df.Emotions.replace({1:'neutral',2:'calm',3:'happy',4:'sad',5:'angry',6:'fear',7:'disgust',8:'surprise'},inplace=True)
Ravdess_df

,Emotions,Path
0,neutral,D:\coderscave\Ravdess\audio_speech_actors_01-2...
1,neutral,D:\coderscave\Ravdess\audio_speech_actors_01-2...
2,neutral,D:\coderscave\Ravdess\audio_speech_actors_01-2...
3,neutral,D:\coderscave\Ravdess\audio_speech_actors_01-2...
4,calm,D:\coderscave\Ravdess\audio_speech_actors_01-2...
...,...,...
1435,surprise,D:\coderscave\Ravdess\audio_speech_actors_01-2...
1436,surprise,D:\coderscave\Ravdess\audio_speech_actors_01-2...
1437,surprise,D:\coderscave\Ravdess\audio_speech_actors_01-2...
1438,surprise,D:\coderscave\Ravdess\audio_speech_actors_01-2...


In [ ]:
datapath=pd.concat([Ravdess_df],axis=0)
datapath.to_csv("data_path.csv",index=False)
datapath.head()

,Emotions,Path
0,neutral,D:\coderscave\Ravdess\audio_speech_actors_01-2...
1,neutral,D:\coderscave\Ravdess\audio_speech_actors_01-2...
2,neutral,D:\coderscave\Ravdess\audio_speech_actors_01-2...
3,neutral,D:\coderscave\Ravdess\audio_speech_actors_01-2...
4,calm,D:\coderscave\Ravdess\audio_speech_actors_01-2...


In [ ]:
def extractFeature(filename,mfcc,chroma,mel):
    with soundfile.SoundFile(filename) as soundFile:
        X=soundFile.read(dtype="float32")
        sampleRate=soundFile.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X,sr=sampleRate,n_mfcc=40).T,axis=0)
            result=np.hstack((result,mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft,sr=sampleRate).T,axis=0)
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X,sr=sampleRate).T,axis=0)
            result=np.hstack((result,mel))
    return result

In [ ]:
emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}
observed=['calm','happy','fearful','disgust']

In [ ]:
def loadData():
    x,y=[],[]
    for file in glob.glob("D:\\coderscave\\Ravdess\\audio_speech_actors_01-24\\Actor_*\\*.wav"):
        fileName=os.path.basename(file)
        emotion1=emotions[fileName.split("-")[2]]
        if emotion1 not in observed:
            continue
        feature=extractFeature(file,mfcc=True,chroma=True,mel=True)
        x.append(feature)
        y.append(feature)
    return train_test_split(np.array(x),y,test_size=0.2,random_state=9)


In [ ]:
x_train,x_test,y_train,y_test=load_data(test_size=0.2)

/content/Actor_01
/content/Actor_13
/content/Actor_03
/content/Actor_04
/content/Actor_05
/content/Actor_18
/content/Actor_07
/content/Actor_02
/content/Actor_14
/content/Actor_21
/content/Actor_08
/content/Actor_15
/content/Actor_17
/content/Actor_10
/content/Actor_16
/content/Actor_11
/content/Actor_09
/content/Actor_06
/content/Actor_24
/content/Actor_20
/content/Actor_23
/content/Actor_12
/content/Actor_19
/content/Actor_22


In [ ]:
print((x_train.shape[0], x_test.shape[0]))
print(f'Features extracted: {x_train.shape[1]}')

(614, 154)
Features extracted: 180


In [ ]:
model = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (300,), learning_rate = 'adaptive', max_iter = 500)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 75.97%
